In [1]:
#from utils import load_data, MidpointNormalize, equipotential_standard_normal, plot_heatmaps
import sys
wd = '/ceph/ibmi/it/users/zabel/tsne/diss'
sys.path.insert(0, wd)

import pandas as pd
#import seaborn as sns

from tsne_jax import *

from utils import *
import matplotlib.pyplot as plt
#import tikzplotlib

from tueplots.constants.color import palettes

import scipy

plt.rcParams.update(cycler.cycler(color=palettes.tue_plot))
#plt.rcParams.update(fonts.aistats2022_tex(family="serif"))
#plt.rcParams.update(fontsizes.aistats2022())
#plt.rcParams['axes.grid'] = False


In [ ]:
def plot_matrix(M):
  for i, row in enumerate(M):
    s = ''
    for j, col in enumerate(row):
      s = s +'('+str(j)+','+str(i)+') ['+str(M[i, j])+'] '
    print(s)
    print('\n')

### Load matrices generated in R

In [ ]:
X = pd.read_csv('/ceph/ibmi/it/users/zabel/tsne/diss/datasets/Scoelicolor/results/M145_30_timepoints/f199_full_genes.tsv', header=0, sep='\t', index_col=0)
print(X.shape)
labels_str = X.columns
X = X.transpose()

# Set the top x% threshold (e.g., top 50%)
top_percent = 5

# Calculate variance for each column
variances = X.var()

# Determine the cutoff variance value
cutoff = variances.quantile(1 - top_percent / 100)

# Filter columns with variance in the top x%
X = X.loc[:, variances >= cutoff]
print(X.shape)


#plt.plot(mean.iloc[0:3].var(axis=0))

In [ ]:
labels_int = [int(i) for i, j in enumerate(labels_str)]
labels_unique = ['t'+i for i in labels_str]
labels = labels_unique
print(labels_int)

In [ ]:
from openTSNE import TSNE
#X = scipy.stats.matrix_normal.rvs(mean, cov1, cov2)
X_array = X.values

#X, y = load_data(10)
key = random.PRNGKey(42)

y_guess = random.normal(key, shape=(X_array.shape[0], 2))
#
Y_star = tsne_fwd(X_array, y_guess)
#tsne = TSNE(verbose=True)
#Y_star = tsne.fit(X_array,)

X_flat, X_unflattener = flatten_util.ravel_pytree(np.array(X_array))   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star)

np.save('/ceph/ibmi/it/users/zabel/tsne/diss/datasets/Scoelicolor/results/M145_30_timepoints/tsne_embedding.npy', Y_star)


In [ ]:
KL_divergence_dy(X_flat, Y_flat, X_unflattener, Y_unflattener, perplexity=30.0)

In [ ]:
from matplotlib import colors

#cmap = plt.get_cmap('tab10')
#norm = colors.Normalize(vmin=0, vmax=len(labels_unique)-1)
#color_mapping = {label: cmap(norm(i)) for i, label in enumerate(labels_unique)}

#cmap = palettes.tue_plot
#color_mapping = {label: cmap[i] for i, label in enumerate(labels_unique)}


f = plt.figure()
#for ind, i in enumerate(labels_unique):
#  print(i)
#  print(labels_int[ind])
#  inds = [j == i for j in labels]
#  print(inds)
#  scatter = plt.scatter(Y_star[inds, 0], Y_star[inds, 1], c = labels_int[ind], cmap='rainbow', label=i)
scatter = plt.scatter(Y_star[:, 0], Y_star[:, 1], c = labels_int, cmap='rainbow')
plt.plot(Y_star[:, 0], Y_star[:, 1], c='lightblue')
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')
for i, txt in enumerate(labels):
  #if i==0:
  plt.annotate(txt, (Y_star[i, 0], Y_star[i, 1]))
  #if i==31:
  #  plt.annotate(txt, (Y_star[i, 0], Y_star[i, 1]))
#plt.legend(handles=scatter.legend_elements(num=8)[0], labels=labels)
plt.legend()



In [ ]:
KL_divergence_dy(X_flat, Y_flat, X_unflattener, Y_unflattener, perplexity=30)

In [ ]:
sensitivities = compute_sensitivities(X_flat, Y_flat, X_unflattener, Y_unflattener, 2.33)
np.save('/ceph/ibmi/it/users/zabel/tsne/diss/datasets/Scoelicolor/results/M145_30_timepoints/sensitivities.npy', sensitivities)

In [ ]:
sensitivities.shape

In [ ]:
#import tikzplotlib
px = 1/plt.rcParams['figure.dpi']
cm = 1/2.54
plot_heatmap(sensitivities, figsize=(14*cm, 7*cm), with_cell_lines=False)

#tikzplotlib.save("sensitivities.pgf")

In [ ]:
dy_dx_per_input = np.sum(np.abs(sensitivities), axis=0)
#f = plt.figure()
#plt.plot(np.sum(np.abs(dy_dx), axis=0))
dy_dx_per_input_reshaped = X_unflattener(dy_dx_per_input) + 1e-8
plot_heatmap(dy_dx_per_input_reshaped, with_cell_lines=False)
np.save('/ceph/ibmi/it/users/zabel/tsne/diss/datasets/Scoelicolor/results/M145_30_timepoints/sensitivities_summarized.npy', dy_dx_per_input_reshaped)

In [ ]:
f = plt.figure(figsize=(10, 5))
plt.plot(np.sum(dy_dx_per_input_reshaped, axis=0))


In [ ]:

f = plt.figure(figsize=(10, 5))
plt.plot(np.sum(dy_dx_per_input_reshaped, axis=0))
plt.xlim((147, 158))


In [ ]:
print(X.columns[range(147, 158)])


In [ ]:
a = np.sum(dy_dx_per_input_reshaped, axis=0)
K = 10
out = np.argsort(-a)[:K]
print(out)
print([i for i in out])
for j in [i for i in out]:
  print(X.columns[j])
  plt.plot(X.values[:, j])

In [ ]:
profiles = X.values[:, [149, 229, 151, 230, 150, 369, 231, 312, 308, 1]]
np.save('profiles.npy', profiles)

SCO4879-SCO4882: biosynthesis of teichulosonic acid

predicted ATP transporter of phosphate: SCO4139, SCO4141, SCO4140, SCO4142
SCO4140: pstA, phosphate ABC transport system permease protein
SCO4141: pstPhosphate ABC transport system permease protein
SCO4142: glycoprotein (PstS)

SCO0494 : part of BGC0000325: coelichelin biosynthetic gene cluster

In [ ]:
N = 0.01*np.identity(X.values.shape[0])
D = np.identity(X.values.shape[1])*np.random.random(X.values.shape[1])
print(D)
cov_final = compute_cov_without_kronecker(X_flat, Y_flat, X_unflattener, Y_unflattener, np.random.random(X.values.shape[0]*X.values.shape[1]), 30.0)
cov_final = cov_final + 1e-3*np.eye(len(cov_final))

In [ ]:
plot_heatmap(cov_final, figsize=(4, 4), with_cell_lines=False)

In [ ]:
from utils import equipotential_standard_normal

n_samples = 15
y_int = [i for i in range(Y_star.shape[0])]
S = equipotential_standard_normal(2 * Y_star.shape[0], n_samples)
L, lower = jax.scipy.linalg.cho_factor(cov_final, lower=True)
samples = np.transpose(np.transpose(np.dot(L, S))+Y_flat)
print(samples.shape)

fig = plt.figure(1)
for j, sample in enumerate(samples.T):
    sample = Y_unflattener(sample)
    for i, s in enumerate(sample): 
        plt.plot(s[0], s[1], alpha=[i for i in np.arange(0.1, 1, 1/(n_samples+2))][j], marker='.')
               #marker=list(Line2D.markers.keys())[j])
#plt.axis('equal')
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')

In [ ]:
#np.save('/ceph/ibmi/it/users/zabel/tsne/diss/datasets/Scoelicolor/results/M145_30_timepoints/samples_for_animation.npy', samples)

In [ ]:
from animate_ifd import *
animate(samples, labels_int, '/ceph/ibmi/it/users/zabel/tsne/diss/datasets/Scoelicolor/results/M145_30_timepoints/animation.gif', cmap='rainbow')

In [ ]:
import matplotlib.pyplot as plt
# Get the tab20 colormap
tab20 = plt.get_cmap('tab20')

# Extract colors as a list of RGB tuples
colors = [tab20(i) for i in range(tab20.N)]

n_samples = 20
y_int = [i for i in range(Y_star.shape[0])]
S = equipotential_standard_normal_within_one_std(2 * Y_star.shape[0], n_samples)
L, lower = jax.scipy.linalg.cho_factor(cov_final, lower=True)
samples = np.transpose(np.transpose(np.dot(L, S))+Y_flat)
print(samples.shape)

fig = plt.figure()
for j, sample in enumerate(samples.T):
    sample = Y_unflattener(sample)
    for i, s in enumerate(sample): 
        plt.plot(s[0], s[1], c=colors[i], alpha=[i for i in np.arange(0.1, 1, 1/(n_samples+2))][j], marker='.')
               #marker=list(Line2D.markers.keys())[j])
#plt.axis('equal')
plt.xlabel('TSNE 1')
plt.ylabel('TSNE 2')

In [ ]:
plt.scatter(dy_dx_per_input, flatten_vars)
gene_names_subset = np.array(np.repeat(X.columns, 8)).reshape(396, 8).flatten("F")
for i, txt in enumerate(gene_names_subset):
    plt.annotate(txt+','+str(i), (dy_dx_per_input[i], flatten_vars[i]))
    plt.xlabel('Sensitivity')
    plt.ylabel('Variance')

In [ ]:
selected_data = pd.read_csv('/local_scratch.old/zabel/tsne/diss/datasets/Scoelicolor/selected_data.txt')
selected_data = selected_data.values
#labels = pd.read_csv('datasets/Thcells/labels.txt', header=None)
#labels = labels.values.flatten().tolist()
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(selected_data.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
selected_data_tsne = tsne_fwd(selected_data, y_guess)